# Generador de código

El requisito: utilizar un modelo Frontier para generar código C++ de alto rendimiento a partir de código Python

<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../resources.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#f71;">Recordatorio: obtenga el código más reciente</h2>
<span style="color:#f71;">Estoy mejorando continuamente estos laboratorios, agregando más ejemplos y ejercicios.
Al comienzo de cada semana, vale la pena verificar que tenga el código más reciente.<br/>
Primero, haga un <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull y combine los cambios según sea necesario</a>. ¿Tiene algún problema? Intente preguntarle a ChatGPT para que le aclare cómo realizar la fusión, o póngase en contacto conmigo.<br/><br/>
Después de haber obtenido el código, desde el directorio llm_engineering, en un indicador de Anaconda (PC) o Terminal (Mac), ejecute:<br/>
<code>conda env update --f environment.yml --prune</code><br/>
O si utilizó virtualenv en lugar de Anaconda, ejecute esto desde su entorno activado en un Powershell (PC) o Terminal (Mac):<br/>
<code>pip install -r requirements.txt</code>
<br/>Luego reinicie el kernel (menú Kernel >> Reiniciar kernel y borrar resultados de todas las celdas) para incorporar los cambios.
</span>
</td>
</tr>
</table>

<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../important.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h1 style="color:#900;">Nota importante</h1>
<span style="color:#900;">
En este laboratorio, utilizo GPT-4o y Claude-3.5-Sonnet, que son los modelos con un precio ligeramente superior. Los costos siguen siendo bajos, pero si prefiere mantenerlos muy bajos, realice los cambios sugeridos a los modelos (3 celdas más abajo).
</span>
</td>
</tr>
</table>

In [2]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [3]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [4]:
# initialize
# NOTA: opción para usar modelos de costo ultra bajo descomentando las últimas 2 líneas

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# ¿Quieres mantener los costos muy bajos? Elimina los comentarios de estas líneas:
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

In [5]:
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [6]:
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [8]:
# creará la salida en un fichero llamado optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [9]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [10]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
exec(pi)

Result: 3.141592658589
Execution Time: 7.899836 seconds


In [13]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function to perform calculation
double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    // Start timer
    auto start_time = std::chrono::high_resolution_clock::now();
    
    // Perform the calculation
    double result = calculate(100000000, 4, 1) * 4;
    
    // End timer
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;
    
    // Output the results with high precision
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

   

In [ ]:
exec(pi)

# Compilación y ejecución de C++

La siguiente celda contiene el comando para compilar un archivo C++ en mi Mac M2.
Compila el archivo `optimized.cpp` en un ejecutable llamado `optimized`

Luego ejecuta el programa llamado `optimized`

Puedes buscar en Google (o preguntarle a ChatGPT) cómo hacer esto en tu plataforma y luego reemplazar las líneas a continuación.
Si no te sientes cómodo con este paso, puedes omitirlo con seguridad: te mostraré exactamente cómo funciona en mi Mac.

In [14]:
# Compila C++ y corre el ejecutable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.199259875000 seconds


In [15]:
optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, long long param1, long long param2) {
    double result = 1.0;
    #pragma omp parallel for reduction(-:result)
    for (long long i = 1; i <= iterations; ++i) {
        double j1 = static_cast<double>(i) * param1 - param2;
        double j2 = static_cast<double>(i) * param1 + param2;
        result -= 1.0 / j1;
        result += 1.0 / j2;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100'000'000LL, 4LL, 1LL) * 4.0;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;
    
    return 0;
}

In [16]:
# Repetimos para Claude: nuevamente, use el enfoque correcto para su plataforma.

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.196161000000 seconds


In [12]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [18]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 23.391921 seconds


In [19]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>

class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 4294967296)
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t value, a, c, m;
};

int max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    int max_sum = std::numeric_limits<int>::min();
    for (int i = 0; i < n; ++i) {
        int current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

int total_max_subarray_sum(int n, uint32_t initial_

In [20]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

optimized.cpp:9:84: warning: implicit conversion from 'long' to 'uint32_t' (aka 'unsigned int') changes value from 4294967296 to 0 [-Wconstant-conversion]
    9 |     LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 4294967296)
      |                                                                                  ~ ^~~~~~~~~~
1 warning generated.
Total Maximum Subarray Sum (20 runs): 0
Execution Time: 0.347725 seconds


In [21]:
optimize_claude(python_hard)

#include <iostream>
#include <vector>
#include <chrono>
#include <limits>
#include <cstdint>
#include <iomanip>

class LCG {
private:
    uint64_t state;
    static constexpr uint64_t a = 1664525;
    static constexpr uint64_t c = 1013904223;
    static constexpr uint64_t m = 1ULL << 32;

public:
    LCG(uint64_t seed) : state(seed) {}

    uint64_t next() {
        state = (a * state + c) % m;
        return state;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    int64_t current_sum = 0;
    int64_t min_sum = 0;

    for (int i = 0; i < n; ++i) {
        int64_t num = static_cast<int64_t>(lcg.next() % (max_val - min_val + 1) + min_val);
        current_sum += num;
        max_sum = std::max(max_sum, current_sum - min_sum);
        min_sum = std::min(min_sum, current_sum);
    }

    return max_sum;
}

int64_t total_max_subarray_sum(int n, uint64_t initial_seed, int min

In [22]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.000542 seconds


In [13]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [14]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [15]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Modelo desconocido")
    for stream_so_far in result:
        yield stream_so_far        

In [16]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="Código en C++:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Seleeciona el modelo", value="GPT")
        convert = gr.Button("Convierte el código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [28]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [29]:
# Deberás cambiar el código en el bloque try para compilar el código C++ para tu plataforma
# Pegué esto en la interfaz de chat de Claude con una solicitud para que me diera una versión para una PC Intel,
# y respondió con algo que parece perfecto: puedes probar un enfoque similar para tu plataforma.

# Versión M2 para Mac para compilar y ejecutar código C++ optimizado:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m2", "-mcpu=apple-m2", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"Ha ocurrido un error:\n{e.stderr}"

In [30]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [1]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código desde Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Código Python:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="Código C++:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convierte el código")
    with gr.Row():
        python_run = gr.Button("Ejecuta Python")
        cpp_run = gr.Button("Ejecuta C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

NameError: name 'gr' is not defined